In [126]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from IPython.display import Image

In [127]:
data = pd.read_csv("jester-data-1.csv", header=None)


In [128]:
data.shape


(24983, 101)

In [129]:
data = data.drop(data.columns[[0]], axis=1)

In [130]:
training = data[:19985]

In [131]:
training.shape

(19985, 100)

In [132]:
test = data[22484:]

In [133]:
test.shape

(2499, 100)

In [134]:
validation = data[19986:22483]

In [135]:
validation.shape

(2497, 100)

In [136]:
data.iloc[1].shape

(100,)

In [137]:
n_features = 2

user_ratings = training.values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_features))
latent_item_features = np.random.random((user_ratings.shape[1],n_features))

In [138]:
latent_item_features

array([[ 0.8316447 ,  0.36093973],
       [ 0.31053412,  0.24496984],
       [ 0.73642789,  0.74109332],
       [ 0.57953406,  0.77796164],
       [ 0.38767173,  0.12882397],
       [ 0.90491437,  0.33011513],
       [ 0.47183819,  0.51406667],
       [ 0.43182037,  0.57794694],
       [ 0.55799509,  0.14893808],
       [ 0.20739438,  0.34988529],
       [ 0.41902098,  0.12371396],
       [ 0.43911174,  0.07532317],
       [ 0.74936004,  0.30377957],
       [ 0.53109086,  0.78911194],
       [ 0.305201  ,  0.5206222 ],
       [ 0.07943023,  0.45589281],
       [ 0.24993273,  0.2470714 ],
       [ 0.16870086,  0.4185225 ],
       [ 0.34402597,  0.62606631],
       [ 0.28769625,  0.82413977],
       [ 0.91121587,  0.92497685],
       [ 0.56586342,  0.31671937],
       [ 0.01727673,  0.99080478],
       [ 0.86117577,  0.7938036 ],
       [ 0.23144642,  0.79501809],
       [ 0.1111538 ,  0.87779765],
       [ 0.92578795,  0.65785116],
       [ 0.539056  ,  0.37747893],
       [ 0.97119339,

In [139]:
latent_user_preferences.shape

(19985, 2)

In [140]:
latent_item_features.shape

(100, 2)

In [141]:
def predict_rating(user_id,item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating,alpha = 0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  ( prediction_rating- rating );
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    


def sgd(iterations = 300000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0,iterations):
        error = []
        for user_id in range(0,latent_user_preferences.shape[0]):
            for item_id in range(0,latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                #if(not np.isnan(rating)):
                if(rating != 99):
                    err = train(user_id,item_id,rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        #if(iteration%10000 == 0 ):
        print mse

In [142]:
user_ratings[5][5]


-4.3200000000000003

In [143]:
#predicted score without training
predictions = latent_user_preferences.dot(latent_item_features.T)
    

In [144]:
predictions

array([[ 0.28653335,  0.14274539,  0.39048189, ...,  0.41953358,
         0.1244713 ,  0.315401  ],
       [ 0.24843253,  0.15128305,  0.44381296, ...,  0.53337529,
         0.05845562,  0.46052715],
       [ 0.38313329,  0.15033834,  0.36710183, ...,  0.31113438,
         0.23928904,  0.14621189],
       ..., 
       [ 0.6827649 ,  0.30501634,  0.7961157 , ...,  0.78317711,
         0.35973073,  0.51278828],
       [ 0.92122357,  0.45282468,  1.23205104, ...,  1.31115858,
         0.41116809,  0.97249307],
       [ 0.52369669,  0.24889318,  0.66777637, ...,  0.69279728,
         0.2490702 ,  0.49486875]])

error = []
for row in range(len(training)):
    for col in range(len(training.iloc[row])):
        if training.iat[row,col] != 99:
            error.append(training.iat[row,col] - untrained_preds.iat[row,col])
    
    

 mse = (np.array(error) ** 2).mean()

mse


In [145]:
sgd(iterations=10)

26.067077852
25.0883585658
24.7563748577
24.5096651289
24.2735075107
24.0362998313
23.7964984141
23.5546350347
23.3116841171
23.0685557755


In [146]:
#with alpha@0.01
sgd(iterations=20)

22.8258436534
22.5836463898
22.3414222042
22.0978758565
21.850897403
21.5975914317
21.334456983
21.0577955539
20.7644183499
20.4526619311
20.1235722598
19.7818933898
19.4363136496
19.0984945231
18.7808856895
18.494033526
18.244493139
18.0341499664
17.8609551218
17.7204223836


In [147]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[-2.56948998, -3.64622733, -3.92902287, ..., -1.81919852,
        -3.41611617, -1.79852587],
       [ 2.9688658 ,  3.62773489,  3.73695683, ...,  2.65585078,
         3.24864946,  2.91172529],
       [ 4.48016832,  4.39412433,  4.15739394, ...,  5.03028682,
         3.61308618,  5.93284442],
       ..., 
       [ 1.42541161,  1.20901079,  1.06343193, ...,  1.77934263,
         0.92394981,  2.15686184],
       [ 3.28200478,  3.04886228,  2.81221251, ...,  3.84600381,
         2.44379611,  4.58850302],
       [ 1.26665277,  0.91228481,  0.72267895, ...,  1.73455642,
         0.62762196,  2.14750384]])

In [148]:
values = [zip(user_ratings[i], predictions[i]) for i in range(0,predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = data.columns
comparison_data.applymap(lambda (x,y): "(%2.3f|%2.3f)"%(x,y))

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,(-7.820|-2.569),(8.790|-3.646),(-9.660|-3.929),(-8.160|-6.683),(-7.520|-2.257),(-8.500|-1.614),(-9.850|-3.568),(4.170|-2.912),(-8.980|-5.787),(-4.760|-2.260),...,(2.820|-0.298),(99.000|-1.351),(99.000|0.391),(99.000|-2.204),(99.000|-1.805),(99.000|-1.331),(-5.630|-1.036),(99.000|-1.819),(99.000|-3.416),(99.000|-1.799)
1,(4.080|2.969),(-0.290|3.628),(6.360|3.737),(4.370|4.918),(-2.380|2.539),(-9.660|3.059),(-0.730|2.891),(-5.340|2.112),(8.880|4.610),(9.220|2.950),...,(2.820|1.960),(-4.950|2.456),(-0.290|1.794),(7.860|2.829),(-0.190|2.523),(-2.140|2.506),(3.060|2.298),(0.340|2.656),(-4.320|3.249),(1.070|2.912)
2,(99.000|4.480),(99.000|4.394),(99.000|4.157),(99.000|2.244),(9.030|3.703),(9.270|6.820),(9.030|2.091),(9.270|0.874),(99.000|3.124),(99.000|5.078),...,(99.000|5.940),(99.000|5.357),(99.000|6.851),(9.080|4.791),(99.000|4.617),(99.000|5.568),(99.000|5.499),(99.000|5.030),(99.000|3.613),(99.000|5.933)
3,(99.000|2.769),(8.350|2.874),(99.000|2.786),(99.000|2.143),(1.800|2.308),(8.160|3.894),(-2.820|1.624),(6.210|0.878),(99.000|2.490),(1.840|3.048),...,(99.000|3.236),(99.000|3.070),(99.000|3.630),(0.530|2.885),(99.000|2.736),(99.000|3.181),(99.000|3.103),(99.000|2.960),(99.000|2.422),(99.000|3.443)
4,(8.500|2.807),(4.610|2.458),(-4.170|2.201),(-5.390|-0.004),(1.360|2.285),(1.600|4.873),(7.040|0.691),(4.610|-0.083),(-0.440|0.913),(5.730|3.352),...,(5.190|4.533),(5.580|3.806),(4.270|5.421),(5.190|3.143),(5.730|3.112),(1.550|3.975),(3.110|3.999),(6.550|3.430),(1.800|1.912),(1.600|4.136)
5,(-6.170|-4.969),(-3.540|-5.924),(0.440|-6.052),(-8.500|-7.523),(-7.090|-4.232),(-4.320|-5.421),(-8.690|-4.529),(-0.870|-3.218),(-6.650|-7.192),(-1.800|-5.024),...,(-3.540|-3.688),(-6.890|-4.336),(-0.680|-3.569),(-2.960|-4.806),(-2.180|-4.334),(-3.350|-4.439),(0.050|-4.124),(-9.080|-4.585),(-5.050|-5.261),(-3.450|-5.084)
6,(99.000|2.898),(99.000|2.842),(99.000|2.689),(99.000|1.450),(8.590|2.395),(-9.850|4.412),(7.720|1.352),(8.790|0.564),(99.000|2.019),(99.000|3.285),...,(99.000|3.842),(99.000|3.465),(99.000|4.432),(99.000|3.099),(99.000|2.986),(2.330|3.602),(99.000|3.557),(99.000|3.254),(99.000|2.337),(99.000|3.838)
7,(6.840|-1.903),(3.160|-2.308),(9.170|-2.371),(-6.210|-3.067),(-8.160|-1.625),(-1.700|-1.997),(9.270|-1.816),(1.410|-1.316),(-5.190|-2.892),(-4.420|-1.902),...,(7.230|-1.306),(-1.120|-1.601),(-0.100|-1.218),(-5.680|-1.822),(-3.160|-1.631),(-3.350|-1.636),(2.140|-1.506),(-0.050|-1.719),(1.310|-2.061),(0.000|-1.892)
8,(-3.790|-0.265),(-3.540|-0.785),(-9.420|-0.967),(-6.890|-2.651),(-8.740|-0.281),(-0.290|0.669),(-5.290|-1.229),(-8.930|-1.177),(-7.860|-2.050),(-1.600|0.005),...,(4.370|1.100),(-0.290|0.487),(4.170|1.612),(-0.290|-0.029),(-0.290|0.121),(-0.290|0.540),(-0.290|0.664),(-0.290|0.200),(-3.400|-0.841),(-4.950|0.398)
9,(3.010|4.494),(5.150|4.824),(5.150|4.742),(3.010|4.249),(6.410|3.764),(5.150|5.990),(8.930|2.974),(2.520|1.769),(3.010|4.612),(8.160|4.852),...,(99.000|4.807),(4.470|4.736),(99.000|5.273),(99.000|4.604),(99.000|4.319),(99.000|4.896),(99.000|4.732),(99.000|4.651),(99.000|4.122),(99.000|5.357)


In [156]:
user_ratings_99_replaced = DataFrame(user_ratings)

In [157]:
user_ratings_99_replaced.shape

(19985, 100)

In [158]:
predictions_df = DataFrame(predictions)

In [162]:
user_ratings_99_replaced.columns


RangeIndex(start=0, stop=100, step=1)

In [161]:
predictions_df.columns

RangeIndex(start=0, stop=100, step=1)

In [163]:
user_ratings_99_replaced[user_ratings == 99] = predictions_df

In [164]:
user_ratings_99_replaced

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-2.569490,-3.646227,-3.929023,-6.683093,-2.257168,-1.613684,-3.567733,-2.911986,-5.787080,-2.259721,...,-0.298146,-1.350920,0.390853,-2.203522,-1.804625,-1.331375,-1.035734,-1.819199,-3.416116,-1.798526
1,4.080000,-0.290000,6.360000,4.370000,-2.380000,-9.660000,-0.730000,-5.340000,8.880000,9.220000,...,2.820000,-4.950000,-0.290000,7.860000,-0.190000,-2.140000,3.060000,0.340000,-4.320000,1.070000
2,4.480168,4.394124,4.157394,2.244053,3.703068,6.820122,2.090767,0.873665,3.123636,5.078479,...,5.939639,5.357022,6.851403,4.790574,4.616634,5.568084,5.499346,5.030287,3.613086,5.932844
3,2.769402,2.873860,2.786123,2.142575,2.307710,3.894153,1.623996,0.877569,2.490228,3.047841,...,3.236412,3.070430,3.630354,2.885122,2.735724,3.181215,3.102856,2.960250,2.421559,3.442802
4,2.806540,2.458373,2.200698,-0.004331,2.284886,4.872835,0.690770,-0.082719,0.912641,3.351962,...,4.533082,3.805683,5.420940,3.143145,3.112352,3.974640,3.998534,3.429666,1.912179,4.135732
5,-6.170000,-3.540000,0.440000,-8.500000,-7.090000,-4.320000,-8.690000,-0.870000,-6.650000,-1.800000,...,-3.540000,-6.890000,-0.680000,-2.960000,-2.180000,-3.350000,0.050000,-9.080000,-5.050000,-3.450000
6,2.897633,2.841721,2.688513,1.450127,2.394998,4.411577,1.351692,0.564498,2.019340,3.284754,...,3.842298,3.465156,4.432281,3.098521,2.986091,3.601697,3.557299,3.253681,2.336519,3.837553
7,6.840000,3.160000,9.170000,-6.210000,-8.160000,-1.700000,9.270000,1.410000,-5.190000,-4.420000,...,7.230000,-1.120000,-0.100000,-5.680000,-3.160000,-3.350000,2.140000,-0.050000,1.310000,0.000000
8,-3.790000,-3.540000,-9.420000,-6.890000,-8.740000,-0.290000,-5.290000,-8.930000,-7.860000,-1.600000,...,4.370000,-0.290000,4.170000,-0.290000,-0.290000,-0.290000,-0.290000,-0.290000,-3.400000,-4.950000
9,4.493825,4.824434,4.741950,4.248705,3.763732,5.990175,2.974088,1.768933,4.612465,4.852227,...,4.806912,4.736004,5.272761,4.603772,4.318550,4.895657,4.731849,4.651025,4.121665,5.357032


In [165]:
joke_ratings_total = user_ratings_99_replaced.sum(axis=0)

In [166]:
joke_ratings_total

0     18869.774298
1     11083.485948
2      8506.151598
3    -19891.655725
4     12586.705165
5     36430.839486
6     -4004.774207
7     -7098.843519
8     -6859.771421
9     25377.294885
10    37163.634202
11    34039.434751
12   -26200.982601
13    33618.433590
14   -26735.120459
15   -52321.372392
16   -13414.716222
17    -7903.102614
18     7598.196281
19    -8918.626367
20    44678.008660
21    20484.132942
22     7043.466786
23   -26574.125383
24    14252.693907
25    32637.715403
26    62581.794946
27    35130.298458
28    59094.004194
29    -1982.187020
          ...     
70    -2451.522197
71    52672.341790
72    26272.078651
73   -19309.579968
74     1767.119268
75    48311.380821
76    19548.212392
77    35765.994652
78    10508.272714
79    27628.052623
80    36764.406648
81    21801.842674
82    41042.509762
83    18287.517677
84    23402.653462
85     6508.276515
86    38399.393053
87    40377.287011
88    66329.313824
89    16680.068114
90    38605.688652
91    27651.

In [167]:
best_joke = joke_ratings_total.idxmax()

In [168]:
best_joke

49

In [169]:
worst_joke = joke_ratings_total.idxmin()

In [170]:
worst_joke

57